In [ ]:
# I have trained the Resnet Model from scratch
# I also loaded the best model I have on the resnet18.pt file, which goes up to 96.24% on test data and 100% on training data

from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)
input_size = (224,224)
batch_size = 32
#from __future__ import print_function, division
#cudnn.benchmark = True
plt.ion()   # interactive mode
criterion = nn.CrossEntropyLoss()
# Data Transformations

data_dir = '../input/vgg-data/z'
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.7043, 0.6402, 0.5553],[0.3503, 0.3684, 0.4062])
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.7043, 0.6402, 0.5553],[0.3503, 0.3684, 0.4062])
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Create training and validation datasets
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
# Create training and validation dataloaders
print(image_datasets['train'][0][0].shape)
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'val']}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class_names = image_datasets['train'].classes

In [ ]:
def get_mean_and_std(dataloader):
    channels_sum, channels_squared_sum, num_batches = 0, 0, 0
    for data, _ in dataloader:
        # Mean over batch, height and width, but not over the channels
        channels_sum += torch.mean(data, dim=[0,2,3])
        channels_squared_sum += torch.mean(data**2, dim=[0,2,3])
        num_batches += 1
    
    mean = channels_sum / num_batches

    # std = sqrt(E[X^2] - (E[X])^2)
    std = (channels_squared_sum / num_batches - mean ** 2) ** 0.5

    return mean, std

In [ ]:
mean, std = get_mean_and_std(dataloaders_dict['train'])

In [ ]:
import matplotlib.pyplot as plt
image_datasets['train'][0][0].shape
plt.imshow(image_datasets['train'][0][0].numpy().transpose(1,2,0))

In [ ]:
import numpy as np
import copy
img = copy.deepcopy(image_datasets['train'][0][0].numpy().transpose(1,2,0))
plt.imshow(img)
def set_pixel(x, i, j, value=0):
    x[0][i][j] = value
    x[1][i][j] = value
    x[2][i][j] = value
    return x



In [ ]:

plt.imshow(set_pixel(img, 100,150))

In [ ]:
inputs = image_datasets['train'][0][0]
inputs.to("cuda")


In [ ]:
img, label = next(iter(dataloaders_dict['train']))


In [ ]:
img = img.to("cuda")
import random

for pic in img:
    random_x = int(random.random()*224)
    random_y = int(random.random()*224)
    #print(pic)
    pic = set_pixel(pic, random_x, random_y)
    


In [ ]:
img = img.to("cpu")
plt.imshow(img[5].numpy().transpose(1,2,0))

In [ ]:
img = img.to("cuda")
outputs = model(img)

In [ ]:
_,preds = np.max(outputs, axis = 1)

In [ ]:
model = models.resnet18(pretrained = True)
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, 18)
model.load_state_dict(torch.load('../input/resnet18/resnet18.pt'))
model.to('cuda')

In [ ]:
x_tests = []
y_tests = []
#x_trains = np.array(x_trains)
for x,y in image_datasets['val']:
    x_tests.append(x.numpy())
    y_tests.append(y)
y_trains = []
x_trains = []
#x_trains = np.array(x_trains)
#for x,y in image_datasets['train']:
#    x_trains.append(x.numpy())
#    y_trains.append(y)

In [ ]:
x_tests = np.array(x_tests)
y_tests = np.array(y_tests)
#x_trains = np.array(x_trains)
#y_trains = np.array(y_trains)

In [ ]:
def one_hot(a, num_classes):
  return np.squeeze(np.eye(num_classes)[a.reshape(-1)])
#y_trains = one_hot(y_trains,18)
y_tests = one_hot(y_tests,18)

In [ ]:
pip install adversarial-robustness-toolbox

In [ ]:
from art.defences.trainer import AdversarialTrainer
from art.attacks.evasion import FastGradientMethod
from art.attacks.evasion import ProjectedGradientDescent
import torch.optim as optim
from art.estimators.classification import PyTorchClassifier

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
optimizer3 = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()
from art.defences.postprocessor import Rounded
post =Rounded()
classy = PyTorchClassifier(
    model=model,
    loss=criterion,
    optimizer = optimizer3,
    input_shape = (3,224,224),
    nb_classes =18,
    clip_values = (0,1)
    #postprocessing_defences = post
    
)


In [ ]:
from art.defences.preprocessor import TotalVarMin
from art.defences.preprocessor import SpatialSmoothing
from art.defences.preprocessor import TotalVarMin
from art.defences.preprocessor import FeatureSqueezing
from art.defences.preprocessor import ThermometerEncoding
from art.defences.preprocessor import GaussianAugmentation
from art.defences.preprocessor import LabelSmoothing
from art.defences.preprocessor import JpegCompression
from art.defences.preprocessor import JpegCompression

In [ ]:
defense = GaussianAugmentation(apply_predict = True, ratio= 2, clip_values= (0,1))

classy3 = PyTorchClassifier(
    model=model,
    loss=criterion,
    optimizer = optimizer,
    input_shape = (3,224,224),
    nb_classes =18,
    clip_values = (0,1)
    #preprocessing_defences = defense
    #post_processing = post
    
)

In [ ]:

predictions = classy.predict(x_tests)
print(predictions)
print(y_tests)
counter = 0
for x in range(479):
    if np.argmax(predictions[x], axis=0) == np.argmax(y_tests[x], axis=0):
        #print(x)
        #print(np.argmax(predictions[x], axis=0))
        #print(np.argmax(y_tests[x], axis=0))
        counter+=1
accuracy = counter / len(y_tests)
print("Clean accuracy:", accuracy)

predictions = classy.predict(test_images_pgd)
print(predictions)
print(y_tests)
counter = 0
for x in range(479):
    if np.argmax(predictions[x], axis=0) == np.argmax(y_tests[x], axis=0):
        #print(x)
        #print(np.argmax(predictions[x], axis=0))
        #print(np.argmax(y_tests[x], axis=0))
        counter+=1
accuracy = counter / len(y_tests)
print("Adversarial PGD accuracy:", accuracy)

In [ ]:
pgd = ProjectedGradientDescent(
    estimator=classy
    
)

In [ ]:
test_images_pgd = pgd.generate(x=x_tests)


In [ ]:
#score_clean = vulnerable_classifier._model.eval(x=x_tests, y=y_tests)
#score_fgm = vulnerable_classifier._model.eval(x=test_images_fgm, y=y_tests)

predictions = classy3.predict(x_tests)
print(predictions)
print(y_tests)
counter = 0
for x in range(479):
    if np.argmax(predictions[x], axis=0) == np.argmax(y_tests[x], axis=0):
        #print(x)
        #print(np.argmax(predictions[x], axis=0))
        #print(np.argmax(y_tests[x], axis=0))
        counter+=1
accuracy = counter / len(y_tests)
print("Clean accuracy:", accuracy)

predictions = classy3.predict(test_images_pgd)
print(predictions)
print(y_tests)
counter = 0
for x in range(479):
    if np.argmax(predictions[x], axis=0) == np.argmax(y_tests[x], axis=0):
        #print(x)
        #print(np.argmax(predictions[x], axis=0))
        #print(np.argmax(y_tests[x], axis=0))
        counter+=1
accuracy = counter / len(y_tests)
print("Adversarial PGD accuracy:", accuracy)



In [ ]:
print(classy2)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten
from art.attacks.poisoning import PoisoningAttackBackdoor
from art.attacks.poisoning.perturbations import add_pattern_bd
from art.defences.detector.poison import ActivationDefence
from art.estimators.classification import KerasClassifier
from art.utils import load_dataset, to_categorical
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pprint
import json

In [ ]:
tf.compat.v1.disable_eager_execution()


In [ ]:
from art.defences.preprocessor import TotalVarMin
from art.defences.preprocessor import SpatialSmoothing
from art.defences.preprocessor import TotalVarMin
from art.defences.preprocessor import FeatureSqueezing
from art.defences.preprocessor import ThermometerEncoding
from art.defences.preprocessor import GaussianAugmentation
from art.defences.preprocessor import LabelSmoothing
from art.defences.preprocessor import JpegCompression
from art.defences.preprocessor import JpegCompression

In [ ]:
defense3 = TotalVarMin(norm = 1, solver = 'Newton-CG', max_iter = 20) 
test_images_pgd_clean = defense3(test_images_pgd)[0]



In [ ]:
defense = GaussianAugmentation(clip_values = (0,1))

#defense2 = JpegCompression(clip_values = (0,1)) 
defense3 = TotalVarMin(norm = 1, solver = 'CG') 
defense4 = SpatialSmoothing()
#test_images_pgd_clean = defense3(test_images_pgd)[0]
test_images_pgd_cleaner = defense4(test_images_pgd_clean)[0]
test_images_pgd_cleanest = defense(test_images_pgd_cleaner)[0]
#test_images_pgd_cleanestest = defense2(test_images_pgd_cleanest)[0]

In [ ]:
defense2 = ThermometerEncoding(clip_values = (0,1))

In [ ]:
test_images_pgd_cleanestest = defense2(test_images_pgd_cleanest)[0]

In [ ]:
defense3 = SpatialSmoothing() 

test_images_pgd_cleanest = defense3(test_images_pgd_cleaner)[0]


In [ ]:
test_images_pgd_cleanestest= defense4(test_images_pgd_cleanest)[0]

In [ ]:
from art.defences.preprocessor import GaussianAugmentation
from art.defences.preprocessor import LabelSmoothing


In [ ]:
defense2 = SpatialSmoothing()
test_images_pgd_cleaner = defense2(test_images_pgd_cleaned)[0]

In [ ]:


defense3 = GaussianAugmentation(apply_predict = True, ratio= 2, clip_values= (0,1))
test_images_pgd_cleanest = defense3(test_images_pgd_cleaner)[0]



In [ ]:
from art.defences.preprocessor import JpegCompression
defense4 = JpegCompression(clip_values=(0,1), apply_predict = True)
test_images_pgd_cleanestest = defense4(test_images_pgd_cleanest)[0]


In [ ]:
test_images_pgd_cleanestest

In [ ]:
import numpy

numpy.save('pgd_cleaned.txt', test_images_pgd_cleaned)
numpy.save('pgd.txt', test_images_pgd)


In [ ]:
ls

In [ ]:
predictions = classy3.predict(test_images_pgd)
#print(predictions)
#print(y_tests)
counter = 0
for x in range(479):
    if np.argmax(predictions[x], axis=0) == np.argmax(y_tests[x], axis=0):
        #print(x)
        #print(np.argmax(predictions[x], axis=0))
        #print(np.argmax(y_tests[x], axis=0))
        counter+=1
accuracy = counter / len(y_tests)
print("Adversarial accuracy:", accuracy)


predictions = classy3.predict(test_images_pgd_cleaner)
#print(predictions)
#print(y_tests)
counter = 0
for x in range(479):
    if np.argmax(predictions[x], axis=0) == np.argmax(y_tests[x], axis=0):
        #print(x)
        #print(np.argmax(predictions[x], axis=0))
        #print(np.argmax(y_tests[x], axis=0))
        counter+=1
accuracy = counter / len(y_tests)
print("With 1 Adversarial Defense accuracy:", accuracy)

predictions = classy3.predict(test_images_pgd_cleaner)
#print(predictions)
#print(y_tests)
counter = 0
for x in range(479):
    if np.argmax(predictions[x], axis=0) == np.argmax(y_tests[x], axis=0):
        #print(x)
        #print(np.argmax(predictions[x], axis=0))
        #print(np.argmax(y_tests[x], axis=0))
        counter+=1
accuracy = counter / len(y_tests)
print("With 2 Adversarial Defenses accuracy:", accuracy)

predictions = classy3.predict(test_images_pgd_cleanest)
#print(predictions)
#print(y_tests)
counter = 0
for x in range(479):
    if np.argmax(predictions[x], axis=0) == np.argmax(y_tests[x], axis=0):
        #print(x)
        #print(np.argmax(predictions[x], axis=0))
        #print(np.argmax(y_tests[x], axis=0))
        counter+=1
accuracy = counter / len(y_tests)
print("With 3 Adversarial Defenses accuracy:", accuracy)





In [ ]:
model = models.resnet18(pretrained = True)
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, 18)
model.load_state_dict(torch.load('../input/resnet18/resnet18.pt'))
model.to('cuda')
classifier = PyTorchClassifier(model = model, loss = criterion, input_shape = (3,224,224),nb_classes = 18)

Adversarial accuracy: 0.020876826722338204
With 1 Adversarial Defense accuracy: 0.4154488517745303
With 2 Adversarial Defenses accuracy: 0.6910229645093946
With 3 Adversarial Defenses accuracy: 0.8517745302713987

data = {'':20, 'C++':15, 'Java':30,
        'Python':35}
courses = list(data.keys())
values = list(data.values())
  
fig = plt.figure(figsize = (10, 5))
 
# creating the bar plot
plt.bar(courses, values, color ='maroon',
        width = 0.4)
 
plt.xlabel("Courses offered")
plt.ylabel("No. of students enrolled")
plt.title("Students enrolled in different courses")
plt.show()

In [ ]:
img, label = next(iter(dataloaders_dict['val']))

In [ ]:
output = classy.predict(test_images_pgd)
output2 = classy.predict(test_images_pgd_cleanest)
#print(output)
preds = np.argmax(output,axis = 1)
print(preds)
preds = np.argmax(y_tests,axis = 1)

print(preds)
preds = np.argmax(output2,axis = 1)[0:478]
print(preds)

In [ ]:
plt.figure(figsize=(7, 5)) 
import matplotlib.pyplot as plt
acc = [0.67223382045929018*100,.8288100208768268*100, 0.8622129436325678*100, ]
model = ["Original", "Spatial Smoothing","Total Variance Minimization"]
plt.title('Accuracy for Different Defences against FGSM (eps = .2)')
plt.xlabel('Defense')
plt.ylabel('Accuracy')
plt.bar(model, acc)
plt.show()

In [ ]:

plt.figure(figsize=(7, 5)) 
import matplotlib.pyplot as plt
acc = [0.020876826722338204*100,0.4154488517745303*100, 0.6910229645093946*100, 0.8517745302713987*100,0.8434237995824635*100]
model = ["Original", "1","1+2", "1+2+3", "1+2+3+4"]
plt.title('Accuracy for Different Defences against PGD')
plt.xlabel('Defense')
plt.ylabel('Accuracy')
plt.bar(model, acc)
plt.show()
plt.legend("hi")

In [ ]:
classifiered = PyTorchClassifier(model = model, loss = criterion, input_shape =(3,224,224), nb_classes = 18)


In [ ]:
def create_figure_axes(
    nrows, 
    ncols, 
    figsize
    ):
    # Creating a figure and axes
    fig, axes = plt.subplots(
        nrows=nrows, 
        ncols=ncols, 
        figsize=figsize
        )

    # Returning the figure and axes
    return fig, axes

In [ ]:
nrows, ncols = 5, 5

# Creating a plot figure and axes for the samples

fig, axes = create_figure_axes(
    nrows=nrows, 
    ncols=ncols, 
    figsize=(20, 25)
    )

# Defining a counting variable
counter = 0

# Indicating the purpose of each column
axes[0, 0].set_title(
        label="PDG images", 
        pad=25
        )
axes[0, 1].set_title(
        label="Clean images", 
        pad=25
        )
axes[0, 2].set_title(
        label="Total Var Min", 
        pad=25
        )
axes[0, 3].set_title(
        label="Total Var Min + Spatial Smooth.", 
        pad=25
        )
axes[0, 4].set_title(
        label="Left Defenses+Gauss.Aug.", 
        pad=25
        )

# Iterating over the axis rows in our figure
for i in range(nrows):    
        # Plotting the adversarial image,
        # turning off axis ticks,
        # and setting the axis title
        axes[i, 0].imshow(test_images_pgd[counter].transpose(1,2,0))        
        axes[i, 0].set_xticks(ticks=[])
        axes[i, 0].set_yticks(ticks=[])
        

        # Plotting the cleaned image,
        # turning off axis ticks,
        # and setting the axis title
        axes[i, 1].imshow(x_tests[counter].transpose(1,2,0))
        axes[i, 1].set_xticks(ticks=[])
        axes[i, 1].set_yticks(ticks=[])
        
        axes[i, 2].imshow(test_images_pgd_cleaned[counter].transpose(1,2,0))
        axes[i, 2].set_xticks(ticks=[])
        axes[i, 2].set_yticks(ticks=[])
        
        axes[i, 3].imshow(test_images_pgd_cleaner[counter].transpose(1,2,0))
        axes[i, 3].set_xticks(ticks=[])
        axes[i, 3].set_yticks(ticks=[])
        
        axes[i, 4].imshow(test_images_pgd_cleanest[counter].transpose(1,2,0))
        axes[i, 4].set_xticks(ticks=[])
        axes[i, 4].set_yticks(ticks=[])



        
        # Incrementing counter value
        counter += 1

# Showing the plot
plt.show()

In [ ]:
plot_images(
    images=x_trains, 
    labels=y_trains,
    plot_label="Label",
    is_categorical=True,
    nrows=2,
    ncols=5,
    figsize=(25, 10))

In [ ]:
model = models.efficientnet_b7(pretrained=True)
#model_ft.classifier[1].out_features = num_classes
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, 18)
model.load_state_dict(torch.load('../input/95-efficientnet/95-efficientnet.pth'))

In [ ]:
model.classifier[1] = nn.Conv2d(512, 18, kernel_size=(1,1), stride=(1,1))
model.num_classes = 18
model.eval()
model.load_state_dict(torch.load('../input/95squeezenet/95-squeezenet.pth'))


In [ ]:
num_classes =18
import torch.nn as nn
import torch
def set_parameter_requires_grad(model, feature_extracting = True):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

set_parameter_requires_grad(model, True)
model.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
model.num_classes = 18
model.load_state_dict(torch.load('../input/95squeezenet/95-squeezenet.pth'))

In [ ]:
pgd_squeezenet = ProjectedGradientDescent(
    estimator=classy, 
    
    
)

In [ ]:
test_images_pgd_sqnet = pgd_squeezenet.generate(x=x_tests)


In [ ]:
from art.defences.preprocessor import TotalVarMin
from art.defences.preprocessor import SpatialSmoothing
from art.defences.preprocessor import TotalVarMin
from art.defences.preprocessor import FeatureSqueezing
from art.defences.preprocessor import ThermometerEncoding

#defense = SpatialSmoothing(window_size = 4)
defense = TotalVarMin()

# Running the defense on adversarial images
test_images_pgd_cleaned_squeeze = defense(test_images_pgd_sqnet)[0]

In [ ]:
from art.defences.preprocessor import GaussianAugmentation
from art.defences.preprocessor import LabelSmoothing


In [ ]:
defense2 = SpatialSmoothing()
test_images_pgd_cleaner_squeeze = defense2(test_images_pgd_cleaned_squeeze)[0]

In [ ]:
defense3 = GaussianAugmentation(apply_predict = True, ratio= 2, clip_values= (0,1))
test_images_pgd_cleanest_squeeze = defense3(test_images_pgd_cleaner_squeeze)[0]


In [ ]:
from art.defences.preprocessor import JpegCompression
defense4 = JpegCompression(clip_values=(0,1), apply_predict = True)
test_images_pgd_cleanestest_squeeze = defense4(test_images_pgd_cleanest_squeeze)[0]


In [ ]:
predictions = classy.predict(x_tests)
#print(predictions)
#print(y_tests)
counter = 0
for x in range(479):
    if np.argmax(predictions[x], axis=0) == np.argmax(y_tests[x], axis=0):
        #print(x)
        #print(np.argmax(predictions[x], axis=0))
        #print(np.argmax(y_tests[x], axis=0))
        counter+=1
accuracy = counter / len(y_tests)
print("Clean accuracy:", accuracy)



predictions = classy.predict(test_images_pgd_sqnet)
#print(predictions)
#print(y_tests)
counter = 0
for x in range(479):
    if np.argmax(predictions[x], axis=0) == np.argmax(y_tests[x], axis=0):
        #print(x)
        #print(np.argmax(predictions[x], axis=0))
        #print(np.argmax(y_tests[x], axis=0))
        counter+=1
accuracy = counter / len(y_tests)
print("Adversarial accuracy:", accuracy)


predictions = classy.predict(test_images_pgd_cleaned_squeeze)
#print(predictions)
#print(y_tests)
counter = 0
for x in range(479):
    if np.argmax(predictions[x], axis=0) == np.argmax(y_tests[x], axis=0):
        #print(x)
        #print(np.argmax(predictions[x], axis=0))
        #print(np.argmax(y_tests[x], axis=0))
        counter+=1
accuracy = counter / len(y_tests)
print("With 1 Adversarial Defense accuracy:", accuracy)

predictions = classy.predict(test_images_pgd_cleaner_squeeze)
#print(predictions)
#print(y_tests)
counter = 0
for x in range(479):
    if np.argmax(predictions[x], axis=0) == np.argmax(y_tests[x], axis=0):
        #print(x)
        #print(np.argmax(predictions[x], axis=0))
        #print(np.argmax(y_tests[x], axis=0))
        counter+=1
accuracy = counter / len(y_tests)
print("With 2 Adversarial Defenses accuracy:", accuracy)

predictions = classy.predict(test_images_pgd_cleanest_squeeze)
#print(predictions)
#print(y_tests)
counter = 0
for x in range(479):
    if np.argmax(predictions[x], axis=0) == np.argmax(y_tests[x], axis=0):
        #print(x)
        #print(np.argmax(predictions[x], axis=0))
        #print(np.argmax(y_tests[x], axis=0))
        counter+=1
accuracy = counter / len(y_tests)
print("With 3 Adversarial Defenses accuracy:", accuracy)
predictions = classy.predict(test_images_pgd_cleanestest_squeeze)
#print(predictions)
#print(y_tests)
counter = 0
for x in range(479):
    if np.argmax(predictions[x], axis=0) == np.argmax(y_tests[x], axis=0):
        #print(x)
        #print(np.argmax(predictions[x], axis=0))
        #print(np.argmax(y_tests[x], axis=0))
        counter+=1
accuracy = counter / len(y_tests)
print("With 3 Adversarial Defenses accuracy:", accuracy)



